In [0]:
def recall_(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def f1_score (y_true, y_pred):
    precision = precision_(y_true, y_pred)
    recall = recall_(y_true, y_pred)
    return 2 * ((precision * recall)/(precision + recall + K.epsilon()))

In [3]:
from __future__ import print_function

import keras.backend as K
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb

max_features = 20000
# cut texts after this number of words (among top max_features most common words)
maxlen = 80
batch_size = 32

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words = max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen = maxlen)
x_test = sequence.pad_sequences(x_test, maxlen = maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 512, mask_zero = True))
model.add(LSTM(512, dropout = 0.1, recurrent_dropout = 0.1, return_sequences = True))
model.add(LSTM(256, dropout = 0.1, recurrent_dropout = 0.1, return_sequences = False))
model.add(Dense(1, activation = 'sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss = 'binary_crossentropy',
              optimizer = 'adam',
              metrics = [recall_, precision_, f1_score])

print('Train...')
model.fit(x_train, y_train,
          batch_size = batch_size,
          epochs = 6,
          validation_data = (x_test, y_test))
score, prec, recall, f1 = model.evaluate(x_test, y_test,
                            batch_size = batch_size)
print('Test score:', score)
print('Test (precision, recall, f1-score):', prec, recall, f1)

Loading data...
25000 train sequences
25000 test sequences
Pad sequences (samples x time)
x_train shape: (25000, 80)
x_test shape: (25000, 80)
Build model...
Train...


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 25000 samples, validate on 25000 samples
Epoch 1/6
25000/25000 [==============================] - 609s 24ms/step - loss: 0.5562 - recall_: 0.7173 - precision_: 0.7242 - f1_score: 0.6906 - val_loss: 0.4908 - val_recall_: 0.6984 - val_precision_: 0.8442 - val_f1_score: 0.7584
Epoch 2/6
25000/25000 [==============================] - 600s 24ms/step - loss: 0.3692 - recall_: 0.8293 - precision_: 0.8559 - f1_score: 0.8316 - val_loss: 0.3782 - val_recall_: 0.8434 - val_precision_: 0.8270 - val_f1_score: 0.8305
Epoch 3/6
25000/25000 [==============================] - 606s 24ms/step - loss: 0.2528 - recall_: 0.9007 - precision_: 0.8980 - f1_score: 0.8954 - val_loss: 0.3996 - val_recall_: 0.7778 - val_precision_: 0.8647 - val_f1_score: 0.8138
Epoch 4/6
25000/25000 [==============================] - 597s 24ms/step - loss: 0.1754 - recall_: 0.9337 - precision_: 0.9370 - f1_score: 0.9327 - val_loss: 0.5138 - val_recall_: 0.8077 - val_precision_: 0.8481 - val_f1_score: 0.8222
Epoch 5/6
2500